In [3]:
!pip install trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.1/244.1 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.0/102.0 kB 62.2 MB/s eta 0:00:00


In [ ]:
## reference https://github.com/huggingface/trl/blob/main/examples/research_projects/toxicity/scripts/gpt-j-6b-toxicity.py

In [1]:
import torch
from torch.optim import Adam
from tqdm import tqdm
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    HfArgumentParser,
    RobertaForSequenceClassification,
    RobertaTokenizer,
)

In [4]:
from trl import AutoModelForCausalLMWithValueHead, PPOConfig, PPOTrainer, create_reference_model, set_seed
from trl.core import LengthSampler

In [11]:
model_name = "allenai/OLMo-1B"

In [12]:
config = PPOConfig(
    model_name = model_name,
    learning_rate = (1.47e-5) * 2,
    ppo_epochs=100,
    mini_batch_size = 8,
    batch_size = 32,
    gradient_accumulation_steps=4
)

In [14]:
config.ppo_epochs

100

In [ ]:
def build_dataset(dataset_name="allenai/real-toxicity-prompts", input_min_text_length=5, input_max_text_length=10
):
    tokenizer = AutoTokenizer.from_pretrained(config.model_name)
    tokenizer.pad_token = tokenizer.eos_token()

    ds = load_dataset(dataset_name, split="train")

    def filter_ln(sample):
        toxicity = sample["prompt"]["toxicity"]
        return toxicity is not None and toxicity > 0.3

    ds = ds.filter(filter_fn, batched=False)

    input_size = 